In [2]:
import subprocess
import sys

def install_and_import(package):
    try:
        __import__(package)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Instalações das bibliotecas

In [3]:
install_and_import('openai-whisper')
install_and_import('pytelegrambotapi')
install_and_import('python-telegram-bot==13.7')
install_and_import('pyttsx3')
install_and_import('telebot')
install_and_import('langchain')
install_and_import('transformers')
install_and_import('langchain')


# Conectar ao Bot

In [4]:
import telebot

# Altere para o token do seu bot
API_TOKEN = '7082441679:AAHGW6GnFHEW02r46Nk-rn8mL40CMuunJ7A'

bot = telebot.TeleBot(API_TOKEN)

# Converter voz em texto

In [5]:
!pip install openai-whisper
import whisper

def transcrever_audio(audio_path):
    # Carregando o modelo Whisper
    model = whisper.load_model("base")

    # Transcrevendo o áudio
    result = model.transcribe(audio_path)

    # Retornando o texto transcrito
    return result['text']

# Converter texto em voz

In [6]:
!pip install gtts pydub


from gtts import gTTS
from pydub import AudioSegment
import os

def gerar_audio_gtts(texto, arquivo_audio="resposta_audio.ogg"):
    """
    Converte texto em áudio usando gTTS e salva como arquivo .ogg.

    Args:
        texto (str): Texto a ser convertido em fala.
        arquivo_audio (str): Caminho do arquivo de saída no formato .ogg.

    Returns:
        str: Caminho do arquivo gerado.
    """
    try:
        # Gerar áudio no formato MP3 usando gTTS
        tts = gTTS(texto, lang="pt")
        arquivo_temp = "resposta_temp.mp3"
        tts.save(arquivo_temp)

        # Converter o áudio de MP3 para OGG usando pydub
        som = AudioSegment.from_mp3(arquivo_temp)
        som.export(arquivo_audio, format="ogg")

        # Remover arquivo temporário
        os.remove(arquivo_temp)

        return arquivo_audio
    except Exception as e:
        print(f"Erro ao gerar áudio: {e}")
        return None

# DB Vetorial

In [7]:
!pip install faiss-cpu
!pip install sentence-transformers

from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Inicialização do modelo e banco de dados vetorial
modelo = SentenceTransformer('all-MiniLM-L6-v2')

# Textos de exemplo para o banco de dados vetorial
textos = [
    "Aula de Algoritmos e Lógica de Programação com o professor Roberto Martins, sala 201, das 8:00 às 9:30, segunda-feira.",
    "Aula de Estruturas de Dados com a professora Fernanda Lima, sala 202, das 9:30 às 11:00, segunda-feira.",
    "Aula de Banco de Dados I com o professor Marcos Souza, sala 203, das 11:00 às 12:30, segunda-feira.",
    "Aula de Engenharia de Software com a professora Camila Rocha, sala 204, das 13:00 às 14:30, terça-feira.",
    "Aula de Sistemas Operacionais com o professor Gustavo Alves, sala 205, das 14:30 às 16:00, terça-feira.",
    "Aula de Redes de Computadores com a professora Patricia Costa, sala 206, das 16:00 às 17:30, terça-feira.",
    "Aula de Programação Web com o professor Felipe Santos, sala 207, das 18:00 às 19:30, quarta-feira.",
    "Aula de Arquitetura de Computadores com a professora Juliana Rodrigues, sala 208, das 19:30 às 21:00, quarta-feira.",
    "Aula de Desenvolvimento de Aplicações para Dispositivos Móveis com o professor André Silva, sala 209, das 8:00 às 9:30, quinta-feira.",
    "Aula de Segurança da Informação com a professora Beatriz Oliveira, sala 210, das 9:30 às 11:00, quinta-feira.",
    "Aula de Inteligência Artificial com o professor Tiago Pereira, sala 211, das 11:00 às 12:30, quinta-feira.",
    "Aula de Programação Orientada a Objetos com a professora Laura Costa, sala 212, das 13:00 às 14:30, sexta-feira.",
    "Aula de Análise de Sistemas com o professor Fernando Gomes, sala 213, das 14:30 às 16:00, sexta-feira.",
    "Aula de Testes de Software com a professora Amanda Lima, sala 214, das 16:00 às 17:30, sexta-feira.",
    "Aula de Metodologias Ágeis com o professor Ricardo Silva, sala 215, das 18:00 às 19:30, segunda-feira.",
    "Aula de Modelagem de Dados com a professora Carla Martins, sala 216, das 19:30 às 21:00, segunda-feira.",
    "Aula de Programação Funcional com o professor Rafael Oliveira, sala 217, das 8:00 às 9:30, terça-feira.",
    "Aula de Gestão de Projetos de TI com a professora Priscila Souza, sala 218, das 9:30 às 11:00, terça-feira.",
    "Aula de Desenvolvimento de Sistemas Distribuídos com o professor João Pedro, sala 219, das 11:00 às 12:30, quarta-feira."
]

# Converte os textos em embeddings
vetores = modelo.encode(textos)

# Configura o banco de dados vetorial com FAISS
indice = faiss.IndexFlatL2(vetores.shape[1])
indice.add(np.array(vetores).astype('float32'))

# Função para buscar o texto mais relevante
def buscar_resposta_mais_relevante(consulta):
    """
    Busca o texto mais relevante no banco de dados vetorial com base na consulta.

    Args:
        consulta (str): Texto da consulta.

    Returns:
        str: Resposta mais relevante encontrada.
    """
    # Gera o vetor da consulta
    vetor_consulta = modelo.encode([consulta])

    # Realiza a busca pelo texto mais similar
    distancias, indices = indice.search(np.array(vetor_consulta).astype('float32'), k=1)

    # Recupera o texto mais relevante
    resposta_mais_relevante = textos[indices[0][0]]
    distancia = distancias[0][0]

    # Retorna o resultado
    return resposta_mais_relevante, distancia


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 59.3 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Configuração do Chatbot

In [8]:
@bot.message_handler(content_types=['text'])
@bot.message_handler(content_types=['audio', 'voice'])


def handle_audio(message):
    if message.content_type == 'audio':
        file_info = bot.get_file(message.audio.file_id)
    elif message.content_type == 'voice':
        file_info = bot.get_file(message.voice.file_id)

    # Baixar o arquivo de áudio
    downloaded_file = bot.download_file(file_info.file_path)

    # caminho onde o áudio será salvo
    audio_file_path = f"audio_{message.from_user.id}.ogg"

    # Salvar o áudio no servidor
    with open(audio_file_path, 'wb') as new_file:
        new_file.write(downloaded_file)

    # Transcrever o áudio usando a função transcrever_audio
    texto_transcrito = transcrever_audio(audio_file_path)

    # Transcrição do áudio
    texto_transcrito = transcrever_audio(audio_file_path)

    # Buscar a resposta mais relevante usando o texto transcrito
    resposta, distancia = buscar_resposta_mais_relevante(texto_transcrito)

    # Responder ao usuário com o texto transcrito
    '''bot.reply_to(message, f"{resposta}")'''

    # Pegue a mensagem do usuário
    consulta = message.text

    # Simulando a função buscar_resposta_mais_relevante (adicione sua função aqui)
    resposta_mais_relevante = f"{resposta}"

    # Gera o áudio
    arquivo_audio = gerar_audio_gtts(resposta_mais_relevante)

    if arquivo_audio and os.path.exists(arquivo_audio):
        with open(arquivo_audio, 'rb') as audio:
            bot.send_audio(message.chat.id, audio)
        os.remove(arquivo_audio)  # Remove o arquivo após envio
    else:
        bot.reply_to(message, "Desculpe, houve um erro ao gerar o áudio.")




# Ativação do Bot

In [ ]:
bot.polling()

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 108MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper